In [41]:
import numpy as np
import pandas as pd 
import pydicom
import os
import scipy.ndimage
import matplotlib.pyplot as plt
import nibabel as nib
from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from scipy import ndimage
import random
from PIL import Image
# Some constants 
INPUT_FOLDER = '/media/pkao/Dataset/COVID Dataset/Non-covid PNA/'
OUTPUT_FOLDER = '/media/pkao/Dataset/COVID Dataset/2DNon-covid PNA_slices/'

# INPUT_FOLDER = '/media/pkao/Dataset/COVID Dataset/COVID/'
# OUTPUT_FOLDER = '/media/pkao/Dataset/COVID Dataset/ProcessedCOVID/'

# INPUT_FOLDER = '/media/pkao/Dataset/COVID Dataset/test/'
# OUTPUT_FOLDER = '/media/pkao/Dataset/COVID Dataset/ProcessedOthers/'

patients = os.listdir(INPUT_FOLDER)
print(len(patients))

# Load the scans in given folder path
def load_scan(path):
    series = [pydicom.filereader.dcmread(path + '/' + s) for s in os.listdir(path)]
    slices = []
#     print(os.listdir(path))
#     print(slices)
    for i in range(len(series)):
        try:
            if(series[i].ImagePositionPatient[2]):
                slices.append(series[i])
        except:
            continue
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

31


In [34]:
MIN_BOUND = -1000.0
MAX_BOUND = 400.0
    
def normalize(image):
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image>1] = 1.
    image[image<0] = 0.
    return image


In [35]:
n_slices = 10

In [42]:
for i in range(len(patients)):
#     os.mkdir(OUTPUT_FOLDER + patients[i])
    first_patient = load_scan(INPUT_FOLDER + patients[i])
    first_patient_pixels = get_pixels_hu(first_patient)
    first_patient_pixels = normalize(first_patient_pixels)
    print(first_patient_pixels.shape)
    #Generate n_slices random numbers between 10 and 30
    randomlist = random.sample(range(0,first_patient_pixels.shape[0] ), n_slices)
    for slice in randomlist:
        data = np.array(first_patient_pixels[slice])
        #Rescale to 0-255 and convert to uint8
        rescaled = (255.0 / data.max() * (data - data.min())).astype(np.uint8)
        im = Image.fromarray(rescaled)
#         nib.save(mask, os.path.join(OUTPUT_FOLDER + patients[i],'mask.nii.gz'))
        im.save(os.path.join(OUTPUT_FOLDER, patients[i] + '_slice_'+str(slice)+'.png'))
#         im.save('test.jpg')
    
    
#     plt.hist(first_patient_pixels.flatten(), bins=80, color='c')
#     plt.xlabel("Hounsfield Units (HU)")
#     plt.ylabel("Frequency")
#     plt.show()
    
# #     Show some slice in the middle
#     plt.imshow(first_patient_pixels[20], cmap=plt.cm.gray)
#     plt.show()
    
#     plt.hist(first_patient_pixels.flatten(), bins=80, color='c')
#     plt.xlabel("Hounsfield Units (HU)")
#     plt.ylabel("Frequency")
#     plt.show()
    
# #     Show some slice in the middle
#     plt.imshow(first_patient_pixels[20], cmap=plt.cm.gray)
#     plt.show()

#     pix_resampled, spacing = resample(first_patient_pixels, first_patient, [1,1,1])
#     print("Shape before resampling\t", first_patient_pixels.shape)
#     print("Shape after resampling\t", pix_resampled.shape)
#     segmented_lungs = segment_lung_mask(pix_resampled, False)
#     segmented_lungs_fill = segment_lung_mask(pix_resampled, True)
#     # img = normalize(pix_resampled)
#     # img = zero_center(img)
#     img = np.where(segmented_lungs_fill!=0, pix_resampled, 0)
#     # img = np.where(segmented_lungs_fill!=0, img, 0)
#     print(np.flip(img.T, 1).shape)
#     img = nib.Nifti1Image(np.flip(img.T, 1), np.eye(4))
#     os.mkdir(OUTPUT_FOLDER + patients[i])
#     nib.save(img, os.path.join(OUTPUT_FOLDER + patients[i],'masked_ct.nii'))
#     mask = nib.Nifti1Image(np.flip(segmented_lungs_fill.T, 1), np.eye(4))
#     nib.save(mask, os.path.join(OUTPUT_FOLDER + patients[i],'mask.nii.gz'))

(47, 512, 512)
(53, 512, 512)
(130, 512, 512)
(59, 512, 512)
(70, 512, 512)
(49, 512, 512)
(72, 512, 512)
(58, 512, 512)
(111, 512, 512)
(117, 512, 512)
(64, 512, 512)
(52, 512, 512)
(56, 512, 512)
(122, 512, 512)
(73, 512, 512)
(57, 512, 512)
(46, 512, 512)
(67, 512, 512)
(65, 512, 512)
(65, 512, 512)
(109, 512, 512)
(70, 512, 512)
(60, 512, 512)
(65, 512, 512)
(73, 512, 512)
(121, 512, 512)
(106, 512, 512)
(64, 512, 512)
(65, 512, 512)
(55, 512, 512)
(63, 512, 512)


In [49]:
from sklearn.model_selection import train_test_split

path = '/home/pkao/covid/UCSB_COVID/Data-split/COVID'

data = os.listdir('/home/pkao/covid/UCSB_COVID/Images-processed/CT_COVID')
print(len(data))
x_train, x_test = train_test_split(data, test_size=0.2, random_state=15)
x_train, x_valid = train_test_split(x_train, test_size=0.2, random_state=25)
print(len(x_train))
print(len(x_valid))
print(len(x_test))
file1 = open(os.path.join(path + 'testCT_NonCOVID.txt'), "w")
for i in x_test:
    file1.write(str(i) + "\n")
file1.close()

file1 = open(os.path.join(path + 'trainCT_NonCOVID.txt'), "w")
for i in x_train:
    file1.write(str(i) + "\n")
file1.close()

file1 = open(os.path.join(path + 'valCT_NonCOVID.txt'), "w")
for i in x_val:
    file1.write(str(i) + "\n")
file1.close()

path = '/home/pkao/covid/UCSB_COVID/Data-split/NonCOVID/'
data = os.listdir('/home/pkao/covid/UCSB_COVID/Images-processed/CT_NonCOVID')
print(len(data))
x_train, x_test = train_test_split(data, test_size=0.2, random_state=15)
x_train, x_valid = train_test_split(x_train, test_size=0.2, random_state=25)
print(len(x_train))
print(len(x_valid))
print(len(x_test))

file1 = open(os.path.join(path + 'testCT_COVID.txt'), "w")
for i in x_test:
    file1.write(str(i) + "\n")
file1.close()

file1 = open(os.path.join(path + 'trainCT_COVID.txt'), "w")
for i in x_train:
    file1.write(str(i) + "\n")
file1.close()

file1 = open(os.path.join(path + 'valCT_COVID.txt'), "w")
for i in x_val:
    file1.write(str(i) + "\n")
file1.close()
             
# trainCT_COVID.txt
# valCT_COVID.txt

# testCT_NonCOVID.txt
# trainCT_NonCOVID.txt
# valCT_NonCOVID.txt


100
64
16
20
310
198
50
62
